In [ ]:
%pip install pyxlsb
%pip install Unidecode
%pip install gcsfs

In [ ]:
from datetime import date, datetime
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from collections import namedtuple
from google.cloud import storage, bigquery
from datetime import timedelta
from functools import reduce
from typing import List
from pyspark.sql.types import StructType, StructField, StringType, DateType
from time import sleep
import unidecode
import re
import pandas as pd


In [ ]:
# CURRENT_DATE_ARG = "2024-12-30T23:59:59"
CURRENT_DATE_ARG = None
CURRENT_DATE_ARG

In [ ]:
PARENT_PROJECT = "cloud-macro-tim"
BUCKET_NAME = "tim-ultrafibra-gcs-sp"
ORIGINS=["ultrafibra-base-agendamento"]
CURRENT_DATE = datetime.strptime(CURRENT_DATE_ARG, '%Y-%m-%dT%H:%M:%S') if CURRENT_DATE_ARG is not None else datetime.today()
DATASET_BY_ORIGIN = { 
  "ultrafibra-base-agendamento": "disparos_agendamento",
}

In [ ]:
BRAZILIAN_TIMEDIFF = timedelta(hours=3)
CURRENT_DATE = CURRENT_DATE# - BRAZILIAN_TIMEDIFF
CURRENT_DATE

In [ ]:
def is_midnight_hour(hour):
  return hour >= 0 and hour <= 2

if is_midnight_hour(CURRENT_DATE.hour):
  CURRENT_DATE = CURRENT_DATE - timedelta(days=1)
  CURRENT_DATE = CURRENT_DATE.replace(hour=23, minute=59, second=59)

In [ ]:
# spark = SparkSession.builder.appName("raw_to_refined")\
#   .config('parentProject', PARENT_PROJECT)\
#   .config("spark.sql.caseSensitive", "True")\
#   .config('spark.sql.session.timeZone', 'America/Sao_Paulo')\
#   .config("spark.jars", "gs://spark-lib/bigquery/spark-3.3-bigquery-0.34.0.jar")\
#   .getOrCreate()

# gcsClient = storage.Client()
# bqClient = bigquery.Client()


import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/work/Old/tim_ultrafibra/cloud-macro-tim-7291d4c1fbb9.json"
MAX_MEMORY = "5g"
spark = SparkSession.builder \
    .appName("Ler Parquet do GCS") \
    .config("spark.sql.caseSensitive", "True")\
    .config("spark.sql.session.timeZone", "America/Sao_Paulo")\
    .config("spark.network.timeout", "600s")\
    .config("spark.executor.heartbeatInterval", "120s")\
    .config("spark.jars", 
            "/usr/local/spark/jars/gcs-connector-hadoop3-latest.jar,"
            "/usr/local/spark/jars/hadoop-common-3.3.6.jar,"
            "/usr/local/spark/jars/hadoop-client-api-3.3.4.jar,"
            "/usr/local/spark/jars/hadoop-client-runtime-3.3.4.jar") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", os.environ["GOOGLE_APPLICATION_CREDENTIALS"]) \
    .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.jars", "gs://spark-lib/bigquery/spark-3.3-bigquery-0.34.0.jar")\
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()
gcsClient = storage.Client()
bqClient = bigquery.Client()

In [ ]:
def to_snake_case(input_string):
    input_string = input_string.replace(" ", "_").replace("-", "_")
    snake_case_string = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', input_string)
    return snake_case_string.lower()

def sanitize_column_name(text):
  text = unidecode.unidecode(text)
  text = to_snake_case(text).replace("/", "_")
  text = re.sub(r"_\d+(\.\d?)*_", "_", text, count=1)
  return text

def rename_columns(df: DataFrame) -> DataFrame:
  for col_name in df.columns:
    df = df.withColumnRenamed(col_name, sanitize_column_name(col_name))
  return df

In [ ]:
def get_path_in_raw_zone(datetime: datetime, origin: str):
  return f"gs://{BUCKET_NAME}/raw-zone/{origin}/{datetime.strftime('%Y%m')}/{datetime.strftime('%Y%m%d-%H%M%S')}.csv"

def get_path_in_curated_zone(datetime: datetime, origin: str):
  print(f"gs://{BUCKET_NAME}/curated-zone/{origin}/{datetime.strftime('%Y%m')}/{datetime.strftime('%Y%m%d')}.parquet")
  return f"gs://{BUCKET_NAME}/curated-zone/{origin}/{datetime.strftime('%Y%m')}/{datetime.strftime('%Y%m%d')}.parquet"

def get_path_in_curated_zone_backup(datetime: datetime, origin: str):
  print(f"gs://{BUCKET_NAME}/curated-zone/{origin}/{datetime.strftime('%Y%m')}/{datetime.strftime('%Y%m%d')}.parquet")
  return f"gs://{BUCKET_NAME}/curated-zone/{origin}/backup/{datetime.strftime('%Y%m')}/{datetime.strftime('%Y%m%d')}.parquet"

def get_path_in_refined_zone(datetime: datetime, origin: str, table_name: str):
  print(f"gs://{BUCKET_NAME}/refined-zone/{origin}/{datetime.strftime('%Y%m')}/{table_name}.parquet")
  return f"gs://{BUCKET_NAME}/refined-zone/{origin}/{datetime.strftime('%Y%m')}/{table_name}.parquet"

def get_prefix_in_raw_zone(datetime: datetime, origin: str):
  print('raw zone')
  print(f"raw-zone/{origin}/{datetime.strftime('%Y%m')}/{datetime.strftime('%Y%m%d')}")
  return f"raw-zone/{origin}/{datetime.strftime('%Y%m')}/{datetime.strftime('%Y%m%d')}"

def get_prefix_in_curated(datetime: datetime, origin: str):
  return f"curated-zone/{origin}/{datetime.strftime('%Y%m')}"

def get_prefix_backup_in_curated(origin: str):
  return f"curated-zone/{origin}/backup"


def get_prefix_in_refined(datetime: datetime, origin: str):
  print('refined zone')
  return f"refined-zone/{origin}/{datetime.strftime('%Y%m')}"

def get_tmp_table_name(origin: str):
  display(f'tim_ultrafibra_projeto_tct.{DATASET_BY_ORIGIN[origin]}_tmp')
  return f'tim_ultrafibra_projeto_tct.{DATASET_BY_ORIGIN[origin]}_tmp'

def get_full_table_name(origin: str):
  display(f'tim_ultrafibra_projeto_tct.{DATASET_BY_ORIGIN[origin]}')
  return f'tim_ultrafibra_projeto_tct.{DATASET_BY_ORIGIN[origin]}'

### Raw to Curated

Os dados são lidos da camada `raw` e carregados para a camada `curated` em formato parquet.

In [ ]:
bucket = gcsClient.get_bucket(BUCKET_NAME)

blobs_per_origin_in_raw = {}

for origin in ORIGINS:
  blobs = bucket.list_blobs(prefix=get_prefix_in_raw_zone(CURRENT_DATE, origin))
  blobs_per_origin_in_raw[origin] = [f"gs://{BUCKET_NAME}/{blob.name}" for blob in blobs if blob.name.endswith(".xlsb") and f'{CURRENT_DATE.strftime("%Y%m%d")}' in blob.name]
  blobs_per_origin_in_raw

In [ ]:
for origin_name, paths in blobs_per_origin_in_raw.items():
  final_df = []
  print(origin_name, paths)
  for path in paths:
    df = spark.createDataFrame(pd.read_excel(path, engine="pyxlsb"))
    final_df.append(rename_columns(df))
  if len(final_df) > 1:
    final_df = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), final_df)
    final_df.write.mode("overwrite").parquet(get_path_in_curated_zone(CURRENT_DATE, origin_name))
  elif len(final_df) == 1:
    final_df[0].write.mode("overwrite").parquet(get_path_in_curated_zone(CURRENT_DATE, origin_name))
  

### Curated to Refined

Os blocos a seguir fazem a transformação dos dados curados para os dados refinados. A coluna `montante` é formatada e também removemos duplicatas utilizando as colunas `id_do_registro` e `hora_da_modificacao`.

In [ ]:

blobs_per_origin_in_curated = {}

for origin_name in ORIGINS:
  blobs = gcsClient.list_blobs(BUCKET_NAME, prefix=get_prefix_in_curated(CURRENT_DATE, origin_name))
  blobs_per_origin_in_curated[origin_name] = [f"gs://{BUCKET_NAME}/{blob.name}" for blob in blobs if blob.name.endswith(".parquet/") and CURRENT_DATE.strftime("%Y%m%d") in blob.name]
blobs_per_origin_in_curated

In [ ]:
def per_origin_in_curated():
    """
    Verifica se há arquivos `.parquet/` criados hoje dentro da estrutura curated para cada origem.
    Retorna True se já houver backup (evitando novo disparo), e False caso contrário.
    """
    blobs_per_origin_in_curated = {}

    for origin_name in ORIGINS:
        blobs = gcsClient.list_blobs(BUCKET_NAME, prefix=get_prefix_backup_in_curated(origin_name))
        
        # Filtrar apenas os arquivos da data atual
        arquivos_hoje = [
            f"gs://{BUCKET_NAME}/backup/{blob.name}" for blob in blobs
            if blob.name.endswith(".parquet/") and f"/{CURRENT_DATE.strftime('%Y%m%d')}.parquet/" in blob.name
        ]

        blobs_per_origin_in_curated[origin_name] = arquivos_hoje

    display(blobs_per_origin_in_curated)


    return any(blobs_per_origin_in_curated.values())  # Se houver arquivos, retorna True, senão False

validation = per_origin_in_curated()
print(validation)

In [ ]:
def reduce_dataframe(dfs: list):
  if len(dfs) == 0:
    return spark.createDataFrame([], StringType())
  elif len(dfs) == 1:
    return dfs[0]
  else:
    return reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), dfs)

def save_dataframe(df: DataFrame, origin: str, table_name: str):
  df.write.format("parquet").mode("overwrite").save(get_path_in_refined_zone(CURRENT_DATE,origin, table_name))

def save_dataframe_buckup(df: DataFrame, origin: str):
  df.coalesce(1).write.format("parquet").mode("overwrite").save(get_path_in_curated_zone_backup(CURRENT_DATE,origin))

In [ ]:
rename_columns = {
        'dt_agendamento': 'data_agendamento',
        'hora': 'hora_processada'
}

valid_columns = ["ordem", "uf", 'dt_agendamento', 'status', 'hora']

In [ ]:
def criar_df_vazio():
    """Cria um DataFrame vazio com a estrutura esperada."""
    schema = StructType([
        StructField("dt_agendamento", DateType(), True),
        StructField("ordem", StringType(), True),
        StructField("uf", StringType(), True),
        StructField("status", StringType(), True),
        StructField("hora", StringType(), True),
    ])
    
    return spark.createDataFrame([], schema)

In [ ]:
from pyspark.sql.types import DateType
import requests
import json

def read_file_xlsb():
    for origin in ORIGINS:
        dfs = []
        
        all_ufs = []
        
        paths = blobs_per_origin_in_curated.get(origin, [])
        print(paths)
        if not paths:
            print(f"⚠️ Nenhum arquivo encontrado")
            # df_filtrado = criar_df_vazio()
            # all_ufs = []
            return None, None

        print("🚀 Arquivos Encontrados. Processando dataframe.")
        for path in paths:
            print(f"📂 Processando arquivo: {path}")
            df = spark.read.parquet(path)
            display(df.show())
            dfs.append(df)
        if dfs:
            reduced_df = reduce_dataframe(dfs)

            # all_ufs = reduced_df.select("uf").distinct().rdd.flatMap(lambda x: x).collect()
            all_ufs = (reduced_df.select(F.upper("uf").alias("uf")).distinct().rdd.flatMap(lambda x: x).collect())
            df_spark = (reduced_df.withColumn("uf", F.upper(F.col("uf"))).withColumn("dt_agendamento", F.expr("date_add('1899-12-30', CAST(dt_agendamento AS INT))").cast(DateType())))
            df_spark = df_spark.dropDuplicates()
            # hoje = F.date_sub(F.current_date(), 0)
            # hoje = F.lit("2025-02-19").cast("date")
            hoje = F.current_date()

            df_filtrado = df_spark.filter(F.col("dt_agendamento") == hoje)
            # display(df_filtrado.show())
            return df_filtrado, all_ufs

def send_request(ordens:List[str]):

    if validation:
        print('⚠️ Backup encontrado. Cancelando disparos...')
        return None
    
    
    print("🚀 Nenhum backup encontrado. Iniciando o disparo de mensagens...")
    
    url = "https://www.zohoapis.com/crm/v2/functions/enviarmensagemativabyordernumber/actions/execute?auth_type=apikey&zapikey=1003.a887d07e81c67367e2bd9ded02383d2a.b03cc99897ccf7d0558373c6e68a5cd7"

    data = { "data" : {
        "CellPhoneNumber": None,
        "Orders": ordens,
        "ChatBotStateId": "8bff0e4b-c2bc-4ecf-a221-b7f2a4136b69",
        "ChatBotId": "timliveinicio",
        "ChatBotMessageNamespace": "842a11f072ac_4f6d_b67d_6fc599c69b8b",
        "ChatBotRouterKey": "Key dGltbW92ZWxyb3V0ZXI6azJ3Mk1GU3RWT211eXZYNXZQVXU=",
        "ContactExtrasVariables": {
        "crmId": "49cd0467-a7ec-42b7-8ac5-8fed77fa29f9",
        "cpf": "36610602875"
        },
        "ChatBotMessageName": "tim_tct_agendamento",
        "chatBotMessageNoPayload": None,
        "ChatBotResource": "9c945f28-6433-4de6-9f1e-fb5515c12549",
        "ChatBotMessageYesPayload": None
    }}
    # print('fazendo req...')
    # print(data)
    files = {'arguments': (None, json.dumps(data))}
    response = requests.request("POST", url, files=files)
    # display(response.status_code)
    # display(response.text)

    json_data = response.text
    # display(json_data)
    # json_data = '{"Orders": {"Success": ["1-1655903095358", "1-1656651105927", "1-1656456309233", "1-1656704421755", "1-1656659017704", "1-1656390140197", "1-1656717187506", "1-1656297674826", "1-1656722750560", "1-1656725407194", "1-1656055282650", "1-1656739869671", "1-1656729946669", "1-1656732535438", "1-1656734026035", "1-1656738957776", "1-1656746612835", "1-1656748924932", "1-1656768745691", "1-1656776094545", "1-1656502450956", "1-1656538785705", "1-1656544603429", "1-1656553468349", "1-1656571926645", "1-1656570302313", "1-1656579637540", "1-1656589700651", "1-1656596594580", "1-1656612638654", "1-1656615709385", "1-1656645689915", "1-1656632885225", "1-1656642717110", "1-1656642172484", "1-1656641477325", "1-1656643844684"], "Failure": []}, "Result": "success"}'

    return json_data

def gerar_df(json_data, df_spark:DataFrame):

    dfs = []

    data = json.loads(json_data)
    df = pd.DataFrame({
        "ordem": data["Orders"]["Success"],
        "status": "Success",
    })
    if data["Orders"]["Failure"]:
        df_failure = pd.DataFrame({
            "ordem": data["Orders"]["Failure"],
            "status": "Failure"})
        df = pd.concat([df, df_failure], ignore_index=True)


    df_spark_data = spark.createDataFrame(df)
    df_spark = df_spark.filter(F.col("ordem").isin(df_spark_data.select("ordem").rdd.flatMap(lambda x: x).collect()))

    final_dataframe = df_spark.join(df_spark_data, on="ordem", how="inner")

    
    display(final_dataframe.toPandas())
    # final_dataframe = df_spark.join(df_spark_data, on="ordem", how="inner")
    # display(final_dataframe.toPandas())
    # all_ufs.append(final_dataframe)

    # dataframe = reduce_dataframe(all_ufs)

    return final_dataframe

def dataframe_pr(df_spark:DataFrame):
    df_filtrado = df_spark.filter((F.col("uf") == "PR"))
    ordens_pr = df_filtrado.select("ordem").rdd.flatMap(lambda x: x).collect()
    df_ordenado = df_filtrado.orderBy("ordem")
    display(len(ordens_pr))
    # display(df_ordenado.toPandas())
    ordens_divididas_pr = chunk_list(ordens_pr)
    # display(ordens_divididas_pr)
    return df_ordenado, ordens_divididas_pr

def dataframe_all_ufs(df_spark:DataFrame):
    '''
    Disparo geral sem PR.
    '''

    # ufs = ["SP", "RJ", "MG"]
    ufs = df_spark.select("uf").distinct().rdd.flatMap(lambda x: x).collect()
    display(ufs)
    df_filtrado = df_spark.filter(~(F.col("uf") == "PR") & (F.col("uf").isin(ufs)))
    ordens = df_filtrado.select("ordem").rdd.flatMap(lambda x: x).collect()
    print(len(ordens))
    final_df_all = df_filtrado.orderBy("ordem")
    ordens_divididas = chunk_list(ordens)
    display(len(ordens_divididas))
    return ordens_divididas, final_df_all

def chunk_list(lst, chunk_size=50):
    """Divide uma lista em sublistas de tamanho máximo especificado."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]




In [ ]:
df_spark, all_ufs = read_file_xlsb()

if df_spark != None:
    display(df_spark.show())

all_dfs = []

In [ ]:
def processar_dados_pr(df_spark):
    if df_spark is None:
        print("⚠️ Arquivo .xlsb indisponível!")
        return criar_df_vazio()  
        
    print("✅ Executando fluxo.")
    only_pr = []
    hora_execucao = (datetime.now() - timedelta(hours=3)).strftime("%H:%M:%S") # Captura a hora dentro da função
    final_df_all, ordens_divididas = dataframe_pr(df_spark)
    final_df_all.show()

    for ordem in ordens_divididas:
        
        print(ordem)
        json_data = send_request(ordens=ordem)
        
        if json_data is None:  
            print("⚠️ Json Vazio.")
            final_df_all = criar_df_vazio()
            final_df_all.show()
            return final_df_all

        dfs = gerar_df(json_data=json_data, df_spark=final_df_all)
        

        only_pr.append(dfs)

    final_df_pr = reduce_dataframe(only_pr)
    final_df_pr = final_df_pr.select(*[F.col(col) for col in valid_columns if col in final_df_pr.columns])
    final_df_pr = final_df_pr.withColumn("hora", F.lit(hora_execucao))
    final_df_pr.show()
    all_dfs.append(final_df_pr)
    
    return final_df_pr 

final_df_pr = processar_dados_pr(df_spark)



In [ ]:
def processar_dados_ufs(df_spark):
    if df_spark is None:
        print("⚠️ Arquivo .xlsb indisponível!")
        return criar_df_vazio()  # Retorna um DataFrame vazio
    print("Esperando 3min para o disparo...")
    sleep(180)
    print("✅ Executando fluxo.")
    all_ufs = []
    ordens_divididas, final_df_all = dataframe_all_ufs(df_spark)
    display(ordens_divididas)
    print(final_df_all.show())
    for ordem in ordens_divididas:
        hora_execucao = (datetime.now() - timedelta(hours=3)).strftime("%H:%M:%S")
        hora_execucao  # Captura a hora dentro da função
        print(ordem)
        json_data = send_request(ordens=ordem)

        if json_data is None:  # Se não houver dados, retorna DF vazio
            print("⚠️ Json Vazio.")
            final_df_all = criar_df_vazio()
            final_df_all.show()
            return final_df_all

        dfs = gerar_df(json_data=json_data, df_spark=final_df_all)
        dfs = dfs.withColumn("hora", F.lit(hora_execucao))
        all_ufs.append(dfs)
    
    # Reduzindo os DataFrames
    final_df_all = reduce_dataframe(all_ufs)
    final_df_all = final_df_all.select(*[F.col(col) for col in valid_columns if col in final_df_all.columns])
    final_df_all.show()

    all_dfs.append(final_df_all)
    
    return final_df_all


df_final = processar_dados_ufs(df_spark)


In [ ]:

merged_df = reduce_dataframe(all_dfs)
merged_df = merged_df.withColumn("dt_agendamento", F.col("dt_agendamento").cast(StringType()))
print(merged_df.printSchema())
for old_col, new_col in rename_columns.items():
    print(old_col, new_col)
    if old_col in merged_df.columns:  
        merged_df = merged_df.withColumnRenamed(old_col, new_col)

merged_df.show()

In [ ]:

if merged_df.count() > 0: #salvar disparos
    save_dataframe_buckup(df=merged_df, origin=origin)

if merged_df.count() > 0: #salvar df main
    save_dataframe(df=merged_df, origin=origin, table_name="agendamentos_tct")


### Refined to Bigquery

Os blocos a seguir são responsáveis por fazer a carga dos dados refinados para o Bigquery.

Primeiro são lidos os arquivos da camada refined referentes ao mês armazenado na variável `CURRENT_DATE`.

Para cada arquivo lido, temos um dataframe que remove as duplicidades a partir das colunas `id_do_registro` e `hora_da_modificacao`. Em seguida, é feita a carga para o Bigquery utilizando o método `save_on_bigquery`.

O método `save_on_bigquery` recebe como parâmetro o dataframe, o nome da origem dos dados, a lista de colunas que devem ser salvos no bigquery e a lista de colunas que serão usadas para fazer o merge dos dados novos com os existentes no BigQuery. Neste método, se a tabela não existir, ela é criada. Caso contrário, os dados são adicionados a uma tabela temporária, depois são feitos os merges com a tabela existente e então a tabela temporária é excluída.


In [ ]:
blobs_per_origin_in_refined = {}

for origin_name in ORIGINS:
  blobs = gcsClient.list_blobs(BUCKET_NAME, prefix=get_prefix_in_refined(CURRENT_DATE, origin_name))
  blobs_per_origin_in_refined[origin_name] = [f"gs://{BUCKET_NAME}/{blob.name}" for blob in blobs if blob.name.endswith(".parquet/")]
blobs_per_origin_in_refined

In [ ]:
JoinOnField = namedtuple('JoinOnField', ['field_name', 'isNullable'])

In [ ]:
def save_on_bigquery(df: DataFrame, origin: str, fields: list, joinOn: List[JoinOnField]):
  tmp_table_name = get_tmp_table_name(origin)
  display(tmp_table_name)
  full_table_name = get_full_table_name(origin)
  display(full_table_name)
  table = None

  try:
    table = bqClient.get_table(full_table_name)
  except Exception:
    table = None
  
  if table is None:
    df.write.format('bigquery')\
      .option("temporaryGcsBucket", BUCKET_NAME)\
      .option("table", full_table_name)\
      .option("parentProject", PARENT_PROJECT)\
      .mode("overwrite")\
      .save()
  else:
    bqClient.query(f"DROP TABLE IF EXISTS {tmp_table_name}")

    df.write.format("bigquery")\
      .option("temporaryGcsBucket", BUCKET_NAME)\
      .option("table", tmp_table_name)\
      .option("parentProject", PARENT_PROJECT)\
      .mode("append")\
      .save()

    select = ", ".join(fields)
    joinCondition = " AND ".join([
      f"(COALESCE(target.`{field.field_name}`, \"\") = COALESCE(origin.`{field.field_name}`, \"\"))" if field.isNullable 
      else f"target.`{field.field_name}` = origin.`{field.field_name}`" 
      for field in joinOn
    ])
    update = ", ".join([f"target.`{field}` = origin.`{field}`" for field in fields])
    insertFields = ", ".join(fields)
    insertValues = ", ".join([f"origin.`{field}`" for field in fields])

    query = f"""
        MERGE INTO {full_table_name} AS target
        USING (SELECT {select} FROM {tmp_table_name}) AS origin
        ON {joinCondition}
        WHEN MATCHED THEN 
          UPDATE SET {update}
        WHEN NOT MATCHED THEN
          INSERT ({insertFields})
          VALUES ({insertValues})
      """
    bqClient.query(query).result()
    print("Saved on BigQuery: ", full_table_name)
    bqClient.query(f"DROP TABLE {tmp_table_name}")

  pass


In [ ]:
valid_columns = ['data_agendamento', 'ordem', 'uf', 'status', 'processed_at', 'hora_processada']

In [ ]:
if validation == False and merged_df.count() > 0:
    print('⏳ Gravando no BQ.')
    for origin in DATASET_BY_ORIGIN:
        # print(origin)
        df_deals = spark.read.parquet(get_path_in_refined_zone(CURRENT_DATE, origin, "agendamentos_tct"))
        print(df_deals.show())
        df_deals = df_deals.select([column for column in df_deals.columns if column in valid_columns])

        drop_duplicates_by = ["data_agendamento", "ordem"]
        join_fields_by = [
            JoinOnField("data_agendamento", False),
            JoinOnField("ordem", True),
            # JoinOnField("nome_produto", True),
        ]

        if df_deals.count() > 0:
            save_on_bigquery(
            df_deals.dropDuplicates(drop_duplicates_by).where(F.col("ordem").isNotNull()), 
            origin,
            list(df_deals.columns), 
            join_fields_by)
else:
    print('❌ Arquivo Vazio. Cancelando save_on_bigquery()')
